In [1]:
import os
import shutil

import awkward as ak
import dask
import dask_awkward as dak
import mplhep as hep
import uproot
from coffea.dataset_tools import preprocess
from distributed import Client
from matplotlib import pyplot as plt

import egamma_tnp
from egamma_tnp import ElectronTagNProbeFromNTuples
from egamma_tnp.plot import plot_ratio
from egamma_tnp.utils.histogramming import save_hists

/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/egmtnpenv/lib64/python3.9/site-packages/coffea/lookup_tools/evaluator.py:7: FutureWarning: In version 2024.8.0 (target date: 2024-07-31 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.lookup_tools.json_lookup will be removed by August 2024. Please use lumi_tools or correctionlib instead!.
  from coffea.lookup_tools.json_lookup import json_lookup


In [2]:
fileset = {
    "events_Reference" : {
        "files": {
        "/eos/cms/store/group/phys_egamma/ssaumya/Tracker2025Scenarios/2024-10-28/2024/mc/RelValZEE_14/crab_mc_ZEE_Reference/241028_214942/0000/TnPTree_mc_1.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/ssaumya/Tracker2025Scenarios/2024-10-28/2024/mc/RelValZEE_14/crab_mc_ZEE_Reference/241028_214942/0000/TnPTree_mc_2.root": "tnpEleTrig/fitter_tree"
        },
    },
   "events_EOR3_TkDPGv2" : {
        "files": {
        "/eos/cms/store/group/phys_egamma/ssaumya/Tracker2025Scenarios/2024-10-28/2024_EOR3_TkDPGv2/mc/RelValZEE_14/crab_mc_ZEE_EOR3_TkDPGv2/241028_215855/0000/TnPTree_mc_1.root": "tnpEleTrig/fitter_tree"
        },
    },     
    "events_EOR3_TkDPGv6" : {
        "files": {
        "/eos/cms/store/group/phys_egamma/ssaumya/Tracker2025Scenarios/2024-10-28/2024_EOR3_TkDPGv6/mc/RelValZEE_14/crab_mc_ZEE_EOR3_TkDPGv6/241028_215953/0000/TnPTree_mc_1.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/ssaumya/Tracker2025Scenarios/2024-10-28/2024_EOR3_TkDPGv6/mc/RelValZEE_14/crab_mc_ZEE_EOR3_TkDPGv6/241028_215953/0000/TnPTree_mc_2.root": "tnpEleTrig/fitter_tree"
        },
    },   
}

fileset_available, fileset_updated = preprocess(fileset, step_size=500_000, skip_bad_files=True)

In [17]:
hlt_paths = {
    "Ele30": "passHLTEle30WPTightGsfTrackIsoFilter",
    "Ele32": "passHLTEle32WPTightGsfTrackIsoFilter",
    "Ele115": "passHLTEle115CaloIdVTGsfTrkIdTGsfDphiFilter",
    "Ele135": "passHLTEle135CaloIdVTGsfTrkIdTGsfDphiFilter",
    "Ele23Ele12Leg1": "passHLTEle23Ele12CaloIdLTrackIdLIsoVLTrackIsoLeg1Filter",
    "Ele23Ele12Leg2": "passHLTEle23Ele12CaloIdLTrackIdLIsoVLTrackIsoLeg2Filter",
    "DoubleEle33SeededLeg": "passHLTEle33CaloIdLPixelMatchFilter",
    "DoubleEle33UnseededLeg": "passHLTDiEle33CaloIdLPixelMatchUnseededFilter",
}

plateau_cuts = {
    "Ele30": 35,
    "Ele32": 35,
    "Ele115": 120,
    "Ele135": 140,
    "Ele23Ele12Leg1": 25,
    "Ele23Ele12Leg2": 15,
    "DoubleEle33SeededLeg": 35,
    "DoubleEle33UnseededLeg": 35,
}

In [18]:
def runfilter(events):
    dataset = events.metadata["dataset"]
    print(dataset, "no run cut")
    return events

In [19]:
tnp = ElectronTagNProbeFromNTuples(fileset_available, list(hlt_paths.values()), cutbased_id="passingCutBasedTight122XV1", extra_filter=runfilter)

In [20]:
%%time

to_compute = {}

for name, trigger in hlt_paths.items():
    if name == "Ele115" or name == "Ele135":
        egamma_tnp.binning.set(
            "el_pt_bins",
            [
                5,
                10,
                15,
                20,
                22,
                26,
                28,
                30,
                32,
                34,
                36,
                38,
                40,
                45,
                50,
                60,
                80,
                100,
                105,
                110,
                115,
                120,
                125,
                130,
                135,
                140,
                145,
                150,
                200,
                250,
                300,
                350,
                400,
            ],
        )
    else:
        egamma_tnp.binning.set(
            "el_pt_bins",
            [
                5,
                10,
                12,
                14,
                16,
                18,
                20,
                23,
                26,
                28,
                30,
                32,
                34,
                36,
                38,
                40,
                45,
                50,
                60,
                80,
                100,
                150,
                250,
                400,
            ],
        )
    plateau_cut = plateau_cuts[name]
    to_compute[name] = tnp.get_1d_pt_eta_phi_tnp_histograms(
        trigger,
        uproot_options={"allow_read_errors_with_report": True},
        eta_regions_pt={
            "barrel": [0.0, 1.4442],
            "endcap_loweta": [1.566, 2.0],
            "endcap_higheta": [2.0, 2.5],
        },
        plateau_cut=plateau_cut,
    )
    
dak.necessary_columns(to_compute)

events_Reference no run cut
events_EOR3_TkDPGv2 no run cut
events_EOR3_TkDPGv6 no run cut
events_Reference no run cut
events_EOR3_TkDPGv2 no run cut
events_EOR3_TkDPGv6 no run cut
events_Reference no run cut
events_EOR3_TkDPGv2 no run cut
events_EOR3_TkDPGv6 no run cut
events_Reference no run cut
events_EOR3_TkDPGv2 no run cut
events_EOR3_TkDPGv6 no run cut
events_Reference no run cut
events_EOR3_TkDPGv2 no run cut
events_EOR3_TkDPGv6 no run cut
events_Reference no run cut
events_EOR3_TkDPGv2 no run cut
events_EOR3_TkDPGv6 no run cut
events_Reference no run cut
events_EOR3_TkDPGv2 no run cut
events_EOR3_TkDPGv6 no run cut
events_Reference no run cut
events_EOR3_TkDPGv2 no run cut
events_EOR3_TkDPGv6 no run cut
CPU times: user 22.8 s, sys: 109 ms, total: 22.9 s
Wall time: 23.7 s


{'from-uproot-0bfa8dab96ef21cb37942fb73a968249': frozenset({'el_eta',
            'el_phi',
            'el_pt',
            'el_q',
            'pair_mass',
            'passHLTDiEle33CaloIdLPixelMatchUnseededFilter',
            'passHLTEle115CaloIdVTGsfTrkIdTGsfDphiFilter',
            'passHLTEle135CaloIdVTGsfTrkIdTGsfDphiFilter',
            'passHLTEle23Ele12CaloIdLTrackIdLIsoVLTrackIsoLeg1Filter',
            'passHLTEle23Ele12CaloIdLTrackIdLIsoVLTrackIsoLeg2Filter',
            'passHLTEle30WPTightGsfTrackIsoFilter',
            'passHLTEle32WPTightGsfTrackIsoFilter',
            'passHLTEle33CaloIdLPixelMatchFilter',
            'passingCutBasedTight122XV1',
            'tag_Ele_eta',
            'tag_Ele_pt',
            'tag_Ele_q'}),
 'from-uproot-179c8ddfadc0e22585d8e28210c7f7af': frozenset({'el_eta',
            'el_phi',
            'el_pt',
            'el_q',
            'pair_mass',
            'passHLTDiEle33CaloIdLPixelMatchUnseededFilter',
            'passHLTEle11

In [21]:
%%time

(out,) = dask.compute(to_compute)


CPU times: user 5.79 s, sys: 211 ms, total: 6 s
Wall time: 6.31 s


In [23]:
print(out) 

{'Ele30': ({'events_Reference': {'pt': {'barrel': {'passing': Hist(Variable(array([  5. ,  10. ,  12.5,  15. ,  17.5,  20. ,  22.5,  25. ,  30. ,
        35. ,  40. ,  45. ,  50. ,  60. ,  80. , 100. , 150. , 200. ,
       250. , 300. , 350. , 400. ]), name='pt', label='Pt [GeV]'), storage=Weight()) # Sum: WeightedSum(value=12426, variance=12426), 'failing': Hist(Variable(array([  5. ,  10. ,  12.5,  15. ,  17.5,  20. ,  22.5,  25. ,  30. ,
        35. ,  40. ,  45. ,  50. ,  60. ,  80. , 100. , 150. , 200. ,
       250. , 300. , 350. , 400. ]), name='pt', label='Pt [GeV]'), storage=Weight()) # Sum: WeightedSum(value=3934, variance=3934)}, 'endcap_loweta': {'passing': Hist(Variable(array([  5. ,  10. ,  12.5,  15. ,  17.5,  20. ,  22.5,  25. ,  30. ,
        35. ,  40. ,  45. ,  50. ,  60. ,  80. , 100. , 150. , 200. ,
       250. , 300. , 350. , 400. ]), name='pt', label='Pt [GeV]'), storage=Weight()) # Sum: WeightedSum(value=2337, variance=2337), 'failing': Hist(Variable(array([  5. 

In [24]:
#print(out) 

for dataset in out["Ele30"][1].keys():
    path = f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/TEST/{dataset}"
    if os.path.exists(path):
        shutil.rmtree(path)
    os.mkdir(path)

In [25]:
for name, res in out.items():
    hists, report = res
    for dataset, report_arr in report.items():
        ak.to_json(
            report_arr,
            f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/TEST/{dataset}/{name}_report.json",
            num_readability_spaces=1,
            num_indent_spaces=4,
        )
    for dataset, hs in hists.items():
        save_hists(f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/TEST/{dataset}/{name}_hists.root", hs)

In [26]:
hep.style.use("CMS")
hep.style.use(
    {
        "figure.figsize": (6.4, 4.8),
        "font.size": 14,
        "legend.title_fontsize": 14,
        "savefig.bbox": "tight",
    }
)

In [27]:
def get_histograms(path):
    with uproot.open(path) as file:
        hpt_barrel_pass = file["pt/barrel/passing"].to_hist()
        hpt_barrel_fail = file["pt/barrel/failing"].to_hist()
        hpt_endcap_loweta_pass = file["pt/endcap_loweta/passing"].to_hist()
        hpt_endcap_loweta_fail = file["pt/endcap_loweta/failing"].to_hist()
        hpt_endcap_higheta_pass = file["pt/endcap_higheta/passing"].to_hist()
        hpt_endcap_higheta_fail = file["pt/endcap_higheta/failing"].to_hist()
        hpt_combined_pass = hpt_barrel_pass + hpt_endcap_loweta_pass + hpt_endcap_higheta_pass
        hpt_combined_fail = hpt_barrel_fail + hpt_endcap_loweta_fail + hpt_endcap_higheta_fail

        heta_entire_pass = file["eta/entire/passing"].to_hist()
        heta_entire_fail = file["eta/entire/failing"].to_hist()

        hphi_entire_pass = file["phi/entire/passing"].to_hist()
        hphi_entire_fail = file["phi/entire/failing"].to_hist()

    return (
        hpt_barrel_pass,
        hpt_barrel_fail,
        hpt_endcap_loweta_pass,
        hpt_endcap_loweta_fail,
        hpt_endcap_higheta_pass,
        hpt_endcap_higheta_fail,
        hpt_combined_pass,
        hpt_combined_fail,
        heta_entire_pass,
        heta_entire_fail,
        hphi_entire_pass,
        hphi_entire_fail,
    )

In [28]:
def pt_low_threshold_plot_setup(**legend_kwargs):
    plt.xlim(10, 400)
    plt.ylim(0, 1.2)
    plt.xlabel(r"Offline electron $P_T$ [GeV]")
    plt.ylabel(r"Efficiency")
    plt.xscale("log")
    plt.xticks([10, 100], [10, 100])
    plt.xticks(
        [20, 30, 40, 50, 60, 70, 80, 90, 200, 300, 400],
        [20, 30, 40, 50, None, None, None, None, 200, 300, 400],
        minor=True,
    )
    plt.legend(**legend_kwargs) if legend_kwargs else plt.legend()


def pt_high_threshold_plot_setup(**legend_kwargs):
    plt.xlim(10, 400)
    plt.ylim(0, 1.2)
    plt.xlabel(r"Offline electron $P_T$ [GeV]")
    plt.ylabel(r"Efficiency")
    plt.legend(**legend_kwargs) if legend_kwargs else plt.legend()


def eta_plot_setup(**legend_kwargs):
    plt.xlim(-2.5, 2.5)
    plt.ylim(0, 1.2)
    plt.xlabel(r"Offline electron $\eta$")
    plt.ylabel(r"Efficiency")
    plt.legend(**legend_kwargs) if legend_kwargs else plt.legend()


def phi_plot_setup(**legend_kwargs):
    plt.xlim(-3.32, 3.32)
    plt.ylim(0, 1.2)
    plt.xlabel(r"Offline electron $\phi$")
    plt.ylabel(r"Efficiency")
    plt.legend(**legend_kwargs) if legend_kwargs else plt.legend()

In [29]:
plateau_cut_dict = {
    "Ele30": 35,
    "Ele32": 35,
    "Ele115": 120,
    "Ele135": 140,
    "Ele23Ele12Leg1": 25,
    "Ele23Ele12Leg2": 15,
    "DoubleEle33SeededLeg": 35,
    "DoubleEle33UnseededLeg": 35,
}

lumis = {
    "2023D": 9.525,
    "2024C": 7.416594091,
    "2024D": 7.889161918,
    "2024Ev1": 6.279524894,
    "2024Ev2": 5.040346798,
    "2024F": 26.313333799,
    "2024G": 17.604084255,
}

In [30]:
for tala in list(hlt_paths.keys()):
    for data_period in ["events_Reference"]:
        for mc_dataset in ["events_EOR3_TkDPGv2","events_EOR3_TkDPGv6"]:
            tocompare = [data_period, mc_dataset]
            run = []
            for folder in tocompare:
                run.append(folder.split("_", 2)[2][3:] if "data" in folder else folder.split("_", 1)[1])
            threshold = tala
            if threshold == "Ele32" or threshold == "Ele30":
                suffix = "WPTight_Gsf"
            elif threshold == "Ele115" or threshold == "Ele135":
                suffix = "CaloIdVT_GsfTrkIdT"
            elif threshold == "DoubleEle33SeededLeg":
                suffix = "CaloIdL_MW\nSeeded leg"
            elif threshold == "DoubleEle33UnseededLeg":
                suffix = "CaloIdL_MW\nUnseeded leg"
            elif threshold == "Ele23Ele12Leg1":
                suffix = "CaloIdL_TrackIdL_IsoVL Leg1"
            elif threshold == "Ele23Ele12Leg2":
                suffix = "CaloIdL_TrackIdL_IsoVL Leg2"
            else:
                raise ValueError("Couldn't find proper trigger name")

            plateau_cut_dict = {
                "Ele30": 35,
                "Ele32": 35,
                "Ele115": 120,
                "Ele135": 140,
                "Ele23Ele12Leg1": 25,
                "Ele23Ele12Leg2": 15,
                "DoubleEle33SeededLeg": 35,
                "DoubleEle33UnseededLeg": 35,
            }
            plateau_cut = plateau_cut_dict[threshold]

            filename = threshold
            threshold = threshold.replace("Leg1", "").replace("Leg2", "").replace("SeededLeg", "").replace("UnseededLeg", "")

            plottype = "pt_high_threshold" if threshold == "Ele115" or threshold == "Ele135" else "pt_low_threshold"
            title = f"HLT_{threshold}_{suffix}"
            lumi = []
            for r in run:
                try:
                    l = lumis[r]
                except KeyError:
                    if r == "2022":
                        l = lumis["2022C"] + lumis["2022D"] + lumis["2022E"] + lumis["2022F"] + lumis["2022G"]
                    elif r == "2023":
                        l = lumis["2023B"] + lumis["2023C"] + lumis["2023D"]
                    else:
                        l = "X"
                if not isinstance(l, str):
                    l = round(l, 1)
                lumi.append(l)

            year = []
            for r in run:
                if "2022" in r:
                    year.append("2022")
                elif "2023" in r:
                    year.append("2023")
                else:
                    year.append("2024")

            #if mc_dataset.startswith("mc_"):
            #    rlabel = f"{lumi[0]} $fb^{{-1}}$, {year[0]} (13.6 TeV)"
            #else:
            #    rlabel = f"{lumi[0]} $fb^{{-1}}$, {year[0]} (13.6 TeV) - {lumi[1]} $fb^{{-1}}$, {year[1]} (13.6 TeV)"
            rlabel = "2024 MC"

            if mc_dataset == "events_EOR3_TkDPGv2":
                eff2_kwargs = {"color": "#5790fc"}
            elif mc_dataset == "events_EOR3_TkDPGv":
                eff2_kwargs = {"color": "#7a21dd"}
            else:
                eff2_kwargs = {"color": "#e42536"}

            (
                hpt_barrel_pass1,
                hpt_barrel_all1,
                hpt_endcap_loweta_pass1,
                hpt_endcap_loweta_all1,
                hpt_endcap_higheta_pass1,
                hpt_endcap_higheta_all1,
                hpt_combined_pass1,
                hpt_combined_all1,
                heta_entire_pass1,
                heta_entire_all1,
                hphi_entire_pass1,
                hphi_entire_all1,
            ) = get_histograms(f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/TEST//{tocompare[0]}/{filename}_hists.root")

            (
                hpt_barrel_pass2,
                hpt_barrel_all2,
                hpt_endcap_loweta_pass2,
                hpt_endcap_loweta_all2,
                hpt_endcap_higheta_pass2,
                hpt_endcap_higheta_all2,
                hpt_combined_pass2,
                hpt_combined_all2,
                heta_entire_pass2,
                heta_entire_all2,
                hphi_entire_pass2,
                hphi_entire_all2,
            ) = get_histograms(f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/TEST//{tocompare[1]}/{filename}_hists.root")

            plot_ratio(
                hpt_barrel_pass1,
                hpt_barrel_all1,
                hpt_barrel_pass2,
                hpt_barrel_all2,
                label1=f"{run[0]}",
                label2=f"{run[1]}",
                denominator_type="failing",
                plottype=plottype,
                figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/TEST//{filename}_{run[0]}_vs_{run[1]}_HLT_eff_barrel_pt.png",
                legend_kwargs={"title": f"{title}\n$0.00 < |\eta| < 1.44$"},
                cms_kwargs={"loc": 1, "rlabel": rlabel},
                eff2_kwargs=eff2_kwargs,
                efficiency_label="L1T + HLT Efficiency",
            )

            plot_ratio(
                hpt_endcap_loweta_pass1,
                hpt_endcap_loweta_all1,
                hpt_endcap_loweta_pass2,
                hpt_endcap_loweta_all2,
                label1=f"{run[0]}",
                label2=f"{run[1]}",
                denominator_type="failing",
                plottype=plottype,
                figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/TEST//{filename}_{run[0]}_vs_{run[1]}_HLT_eff_endcap_loweta_pt.png",
                legend_kwargs={"title": f"{title}\n$1.57 < |\eta| < 2.00$"},
                cms_kwargs={"loc": 1, "rlabel": rlabel},
                eff2_kwargs=eff2_kwargs,
                efficiency_label="L1T + HLT Efficiency",
            )

            plot_ratio(
                hpt_endcap_higheta_pass1,
                hpt_endcap_higheta_all1,
                hpt_endcap_higheta_pass2,
                hpt_endcap_higheta_all2,
                label1=f"{run[0]}",
                label2=f"{run[1]}",
                denominator_type="failing",
                plottype=plottype,
                figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/TEST//{filename}_{run[0]}_vs_{run[1]}_HLT_eff_endcap_higheta_pt.png",
                legend_kwargs={"title": f"{title}\n$2.00 < |\eta| < 2.50$"},
                cms_kwargs={"loc": 1, "rlabel": rlabel},
                eff2_kwargs=eff2_kwargs,
                efficiency_label="L1T + HLT Efficiency",
            )

            plot_ratio(
                hpt_combined_pass1,
                hpt_combined_all1,
                hpt_combined_pass2,
                hpt_combined_all2,
                label1=f"{run[0]}",
                label2=f"{run[1]}",
                denominator_type="failing",
                plottype=plottype,
                figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/TEST//{filename}_{run[0]}_vs_{run[1]}_HLT_eff_combined_pt.png",
                legend_kwargs={"title": f"{title}\n$0.00 < |\eta| < 1.44$ or $1.57 < |\eta| < 2.50$"},
                cms_kwargs={"loc": 1, "rlabel": rlabel},
                eff2_kwargs=eff2_kwargs,
                efficiency_label="L1T + HLT Efficiency",
            )

            plot_ratio(
                heta_entire_pass1,
                heta_entire_all1,
                heta_entire_pass2,
                heta_entire_all2,
                label1=f"{run[0]}",
                label2=f"{run[1]}",
                denominator_type="failing",
                plottype="eta",
                figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/TEST//{filename}_{run[0]}_vs_{run[1]}_HLT_eff_eta.png",
                legend_kwargs={"title": f"{title}\n$0.00 < |\eta| < 2.50$\nProbe electron $P_T> {plateau_cut}$ GeV"},
                cms_kwargs={"loc": 1, "rlabel": rlabel},
                eff2_kwargs=eff2_kwargs,
                efficiency_label="L1T + HLT Efficiency",
            )

            plot_ratio(
                hphi_entire_pass1,
                hphi_entire_all1,
                hphi_entire_pass2,
                hphi_entire_all2,
                label1=f"{run[0]}",
                label2=f"{run[1]}",
                denominator_type="failing",
                plottype="phi",
                figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/TEST//{filename}_{run[0]}_vs_{run[1]}_HLT_eff_phi.png",
                legend_kwargs={"title": f"{title}\n$0.00 < |\eta| < 2.50$\nProbe electron $P_T> {plateau_cut}$ GeV"},
                cms_kwargs={"loc": 1, "rlabel": rlabel},
                eff2_kwargs=eff2_kwargs,
                efficiency_label="L1T + HLT Efficiency",
            )